In [ ]:
import cobra
import pandas as pd
import os

# -----------------------------
# 1. Load the metabolic model
# -----------------------------
model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"

if not os.path.exists(model_path):
    raise FileNotFoundError(f"SBML model not found at {model_path}")

model = cobra.io.read_sbml_model(model_path)
print(f"Model loaded: {len(model.genes)} genes, {len(model.reactions)} reactions")

# -----------------------------
# 2. Set biomass reaction as objective
# -----------------------------
model.objective = "BIOMASS_Ec_SynAuto_1"
growth_wt = model.optimize().objective_value
print(f"Baseline growth rate: {growth_wt:.6f}")

if growth_wt <= 1e-9:
    raise ValueError("Baseline growth is effectively zero. Check model setup or environment.")

# -----------------------------
# 3. Run knockouts
# -----------------------------
results = []
gene_list = [g.id for g in model.genes]

for gene in gene_list:
    with model:
        model.genes.get_by_id(gene).knock_out()
        growth_ko = model.optimize().objective_value
        percent_change = (growth_ko - growth_wt) / growth_wt * 100
        lethal_flag = growth_ko <= 1e-9
        results.append({
            "gene": gene,
            "growth_wt": growth_wt,
            "growth_ko": growth_ko,
            "percent_change": percent_change,
            "lethal_flag": lethal_flag
        })

# -----------------------------
# 4. Create DataFrame and rank
# -----------------------------
df = pd.DataFrame(results)
df = df.sort_values("percent_change")  # most negative change first

# -----------------------------
# 5. Save full knockout results
# -----------------------------
full_csv_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\enhanced_knockouts.csv"
df.to_csv(full_csv_path, index=False)
print(f"✅ Full knockout CSV saved: {full_csv_path}")

# -----------------------------
# 6. Save top impactful knockouts (absolute change)
# -----------------------------
df["abs_change"] = df["growth_wt"] - df["growth_ko"]
top_df = df.sort_values("abs_change", ascending=False).head(10)

top_csv_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\top_impactful_knockouts.csv"
top_df.to_csv(top_csv_path, index=False)
print(f"✅ Top impactful knockouts CSV saved: {top_csv_path}")

# Preview results
df.head()